### pytorch 中使用计算图的方式计算导数
比如我们计算下面这个式子z对a和b的导数
$$ Z = 2*a + ab $$

In [5]:
import torch

# 两种开启梯度计算的方式，这样与a，b有关的tensor将采用图计算的方式
a = torch.tensor(3.)
b = torch.tensor(4., requires_grad=True)
a.requires_grad_(True)

tensor(3., requires_grad=True)

In [7]:
f1 = 2 * a
f2 = a * b
z = f1 + f2

In [8]:
# 开启z的反向传播
z.backward()
print(f'a.grad = {a.grad}')
print(f'b.grad = {b.grad}')

a.grad = 6.0
b.grad = 3.0


#### 如果我们不想让之后所有的计算都产生一个计算图可以用以下方式关闭梯度


In [9]:
with torch.no_grad():
    f3 = a * b
    print(f'f2.requies_grad = {f2.requires_grad}')
    print(f'f3.requies_grad = {f3.requires_grad}')

a1 = a.detach()
print(f'a.requies_grad = {a.requires_grad}')
print(f'a1.requies_grad = {a1.requires_grad}')

f2.requies_grad = True
f3.requies_grad = False
a.requies_grad = True
a1.requies_grad = False


#### 向量求导

In [12]:
x = torch.arange(4.0)
x.requires_grad_(True)
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [13]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [14]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [15]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [19]:
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

#### 将向量移动到计算图之外看做常数不进行传播

In [20]:
x.grad.zero_()
y = x * x
u = y.detach()
z1 = u * x
z1.sum().backward()
print(f'u*x的偏导:{x.grad}')

x.grad.zero_()
z2 = y * x
z2.sum().backward()
print(f'x*x*x的偏导:{x.grad}')


u*x的偏导:tensor([0., 1., 4., 9.])
x*x*x的偏导:tensor([ 0.,  3., 12., 27.])
